# **2. Computational life cycle assessment background**

In life cycle impact assessment (LCIA) the aim is to develop characterisation factors which we will denote mathematically by $Q_{ze}$ as it would belong to a Matrix $\boldsymbol{Q}$ of $z$ impact categories rows per $e$ elmentary flows colums. $Q_{ze}$ aims to characterise the impact caused on the state of a biophysical system $z$ by the emission/extraction of elementary flows denoted $g_e$. An elementary flow $g_e$ is a unique specie/substance/material/resource that is either emitted/extracted to/from biosphere due to activities in the technosphere. In LCA, the goal is first to obtain the full list of elementary flows interacting with a product system. The sum of all characterised elementary flows gives an impact score in absolute terms that represents the change to a specific environmental category due to the emission/extraction of elementary flows (as done in Equation 14).

$$
\Delta X_z = \sum_e Q_{ze} g_e =\sum_e\sum_jQ_{ze} B_{ej} s_j = \sum_e\sum_j\sum_i Q_{ze} B_{ej} A_{ij}^{-1} f_i \tag{14}
$$

It is sometimes (as done in this study) to define the process intensity factors $\Lambda_{zj}$.

$$
\Delta X_z = \sum_j \Lambda_{zj} s_j
$$

> <span style="color:rgb(0, 112, 110); font-weight: bold;">Summary of terms</span>
> - $\Delta X_z$ is the change of state of a biophysical system $z$.
> - $Q_{ze}$ is the characterisation factor of an elementary flow $e$ for a biophysical system $z$.
> - $B_{ej}$ is the biosphere intervention (emission/extraction) of an elementary flow $e$ per process $j$.
> - $A_{ij}$ is the technosphere intervention of a reference flow (product) $i$ per process $j$.
> - $\Lambda_{zj}$ is the impact of a process $j$ on a biophysical system $z$.
> - $g_e$ is is the biosphere intervention (emission/extraction) of an elementary flow $e$.
> - $s_j$ is scale of a process $j$ in the technosphere.
> - $f_i$ is the reference flow (demend for a product) of a process $i$ in the technosphere.
> - $z$ is the index of a biophysical system.
> - $e$ is the index of an elementary flow.
> - $j$ is the index of a process.
> - $i$ is the index of a product.

exemple LCA

premise

example TCM